In [ ]:
# define parameters
    data_dir = args.data_dir
    squad_ver = args.squad_ver

    save_dir = args.save_dir
    num_epochs = args.num_epochs
    batch_size = args.batch_size
    learning_rate = args.learning_rate
    # if args.cuda_device is not None:
    #     cuda_device = int(args.cuda_device)
    # else:
    #     cuda_device = None
    cuda_device = args.cuda_device
    distill = args.distill
    if distill:
        distill_weight = args.distill_weight

    # download data and load
    train_data_path, dev_data_path = download_data(data_dir, squad_ver)
    train_data, dev_data = load_data(train_data_path, dev_data_path, squad_ver, distill)
    train_loader, dev_loader = build_data_loaders(train_data, dev_data, batch_size)

    # load pretrained model
    print("Loading model")
    if args.distill:
        model = BidirectionalAttentionFlowDistill.from_pretrained(distill_weight)
    else:
        bidaf_pred = pretrained.load_predictor("rc-bidaf")
        model = bidaf_pred._model

    # move to gpu if using
    if cuda_device >= 0:
        model.cuda(cuda_device)

    # index with vocab
    print("Indexing")
    tic = time.time()

    train_loader.index_with(model.vocab)
    dev_loader.index_with(model.vocab)

    print("Time elapsed:", time.time()-tic)

    # build trainer
    print("Building trainer")
    trainer = build_trainer(model, save_dir, train_loader, dev_loader, num_epochs, learning_rate, cuda_device)

    # train
    print("Starting training")
    tic = time.time()

    trainer.train()

    print("Finished training")
    print("Time elapsed:", time.time()-tic)

    # evaluate trained model
    print("Evaluating")
    tic = time.time()
    results = evaluate(model, dev_loader, cuda_device, output_file=None, predictions_output_file=None)
    print("Time elapsed:", time.time()-tic)

    pdb.set_trace()